In [1]:
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.preprocessing import StandardScaler


pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('Customer-Churn.csv')
data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [3]:
data.shape

(7043, 16)

In [4]:
data.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [5]:
data['totalcharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')

In [6]:
data.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
totalcharges        float64
dtype: object

In [7]:
data.isnull().values.sum()

11

In [8]:

bool_series = pd.isnull(data["totalcharges"]) 
    
data[bool_series] 

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn,totalcharges
488,Female,0,Yes,Yes,0,No,Yes,No,Yes,Yes,Yes,No,Two year,52.55,,No,NaN
753,Male,0,No,Yes,0,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,20.25,,No,NaN
936,Female,0,Yes,Yes,0,Yes,Yes,Yes,Yes,No,Yes,Yes,Two year,80.85,,No,NaN
1082,Male,0,Yes,Yes,0,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,25.75,,No,NaN
1340,Female,0,Yes,Yes,0,No,Yes,Yes,Yes,Yes,Yes,No,Two year,56.05,,No,NaN
3331,Male,0,Yes,Yes,0,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,19.85,,No,NaN
3826,Male,0,Yes,Yes,0,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,25.35,,No,NaN
4380,Female,0,Yes,Yes,0,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,20.00,,No,NaN
5218,Male,0,Yes,Yes,0,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,19.70,,No,NaN
6670,Female,0,Yes,Yes,0,Yes,No,Yes,Yes,Yes,Yes,No,Two year,73.35,,No,NaN


In [9]:
data['totalcharges'] = data['totalcharges'].fillna(0)

In [10]:
data.isnull().values.sum()

0

In [11]:
data.select_dtypes(np.number)

,SeniorCitizen,tenure,MonthlyCharges,totalcharges
0,0,1,29.85,29.85
1,0,34,56.95,1889.50
2,0,2,53.85,108.15
3,0,45,42.30,1840.75
4,0,2,70.70,151.65
...,...,...,...,...
7038,0,24,84.80,1990.50
7039,0,72,103.20,7362.90
7040,0,11,29.60,346.45
7041,1,4,74.40,306.60


In [12]:
y = data['Churn']

numericals = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'totalcharges']
numericals = data[numericals]

In [13]:
transformer = StandardScaler().fit(numericals)
X_stan = transformer.transform(numericals)
X_stan.shape

(7043, 4)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_stan, y, test_size=0.3, random_state=42)

In [15]:
classification = LogisticRegression(random_state=42, max_iter=10000)
classification.fit(X_train, y_train)
predictions = classification.predict(X_test)
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

          No       0.82      0.92      0.87      1539
         Yes       0.68      0.45      0.54       574

    accuracy                           0.79      2113
   macro avg       0.75      0.69      0.70      2113
weighted avg       0.78      0.79      0.78      2113



In [16]:
classification.score(X_test, y_test)

0.7941315664931378

In [17]:
y.value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

In [19]:
data1 = data

In [20]:
cat_no = data1[data1['Churn'] == 'No']
cat_yes = data1[data1['Churn'] == 'Yes']


cat_no = cat_no.sample(len(cat_yes))
print(cat_no.shape)
print(cat_yes.shape)

data2 = pd.concat([cat_no, cat_yes], axis=0)
#shuffling the data
data2 = data2.sample(frac=1)
data2['Churn'].value_counts()

(1869, 17)
(1869, 17)


Yes    1869
No     1869
Name: Churn, dtype: int64

In [23]:
category_no = data[data['Churn'] == 'No']
category_yes = data[data['Churn'] == 'Yes']

In [24]:
data3 = pd.concat([category_no, category_yes], axis=0)

data3 = data3.sample(frac=1)
print(data3['Churn'].value_counts())

No     5174
Yes    1869
Name: Churn, dtype: int64


In [25]:
data4 = pd.concat([cat_no, cat_yes], axis=0)

data3 = data3.sample(frac=1)
print(data4['Churn'].value_counts())

No     1869
Yes    1869
Name: Churn, dtype: int64


In [26]:
data4.isnull().values.sum()

0

In [27]:
y = data4['Churn']

numericals = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'totalcharges']
numericals = data4[numericals]

In [28]:
transformer = StandardScaler().fit(numericals)
X_stan = transformer.transform(numericals)
X_stan.shape

(3738, 4)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_stan, y, test_size=0.3, random_state=42)

In [30]:
classification = LogisticRegression(random_state=42, max_iter=10000)
classification.fit(X_train, y_train)
predictions = classification.predict(X_test)
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

          No       0.74      0.71      0.73       578
         Yes       0.71      0.74      0.72       544

    accuracy                           0.73      1122
   macro avg       0.73      0.73      0.73      1122
weighted avg       0.73      0.73      0.73      1122



In [31]:
classification.score(X_test, y_test)

0.7254901960784313